# ADPA Error Handling and Debugging

This notebook demonstrates advanced error handling and debugging techniques in ADPA:
- Custom exception handling
- Error recovery strategies
- Debugging tools and techniques
- Logging and tracing
- Error reporting

## Setup

First, let's import the necessary components.

In [ ]:
from adpa import Text2SQL, Database
from adpa.config import Config
from adpa.exceptions import (
    Text2SQLError,
    DatabaseError,
    ValidationError,
    SecurityError
)
from adpa.debugging import (
    Debugger,
    ErrorTracer,
    QueryAnalyzer
)
from adpa.logging import Logger

# Initialize components
config = Config()
config.load_env()

db = Database(config)
text2sql = Text2SQL(config)
debugger = Debugger(config)
tracer = ErrorTracer(config)
analyzer = QueryAnalyzer(config)
logger = Logger(config)

## Custom Exception Handling

Demonstrate how ADPA handles different types of errors.

In [ ]:
# Example error scenarios
error_queries = [
    # Invalid SQL syntax
    "Show me the total sales broken by",
    
    # Missing table
    "Show me all records from nonexistent_table",
    
    # Security violation
    "DROP TABLE employees",
    
    # Invalid data type
    "Show employees where salary = 'high'"
]

for query in error_queries:
    print(f"\nProcessing query: {query}")
    try:
        result = text2sql.execute(query)
    except Text2SQLError as e:
        print(f"Text2SQL Error: {e}")
        print("Error Details:")
        print(f"- Type: {e.error_type}")
        print(f"- Context: {e.context}")
        print(f"- Suggestion: {e.suggestion}")
    except DatabaseError as e:
        print(f"Database Error: {e}")
        print("Error Details:")
        print(f"- SQL State: {e.sqlstate}")
        print(f"- Table: {e.table}")
        print(f"- Column: {e.column}")
    except SecurityError as e:
        print(f"Security Error: {e}")
        print("Error Details:")
        print(f"- Violation: {e.violation_type}")
        print(f"- Policy: {e.policy}")
        print(f"- Required Role: {e.required_role}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

## Error Recovery Strategies

Show how ADPA recovers from errors and provides alternatives.

In [ ]:
# Example recovery scenarios
def handle_query_with_recovery(query):
    print(f"\nProcessing query with recovery: {query}")
    
    try:
        # Attempt query execution
        result = text2sql.execute(query)
        return result
    except ValidationError as e:
        print(f"Validation Error: {e}")
        
        # Attempt to fix the query
        fixed_query = analyzer.suggest_fix(query)
        if fixed_query:
            print(f"Suggested fix: {fixed_query}")
            return text2sql.execute(fixed_query)
    except DatabaseError as e:
        print(f"Database Error: {e}")
        
        # Try alternative data source
        backup_result = db.query_backup(query)
        if backup_result:
            print("Retrieved from backup source")
            return backup_result
    except SecurityError as e:
        print(f"Security Error: {e}")
        
        # Try with reduced permissions
        limited_result = text2sql.execute_with_limited_scope(query)
        if limited_result:
            print("Retrieved with limited scope")
            return limited_result
    
    return None

# Test recovery scenarios
recovery_queries = [
    "Show me sales data from last month",  # Table might be archived
    "Show me sensitive employee data",      # Security restrictions
    "Show me the average salery by dept"    # Typo in query
]

for query in recovery_queries:
    result = handle_query_with_recovery(query)
    if result:
        print("Recovery successful!")
        print(f"Results: {result}")

## Debugging Tools

Demonstrate ADPA's debugging capabilities.

In [ ]:
# Enable debug mode
debugger.enable()

# Set breakpoints
debugger.set_breakpoint("text2sql.translate")
debugger.set_breakpoint("database.execute")

# Example debugging session
with debugger.session() as debug:
    query = "Show me employee salaries by department"
    
    # Analyze query
    debug.analyze_query(query)
    
    # Show query plan
    debug.show_plan()
    
    # Execute with tracing
    with tracer.trace() as trace:
        result = text2sql.execute(query)
    
    # Show trace
    trace.display()
    
    # Show variable values
    debug.print_vars()
    
    # Show execution time
    debug.show_timing()

# Disable debug mode
debugger.disable()

## Logging and Tracing

Show ADPA's logging capabilities.

In [ ]:
# Configure logging
logger.set_level("DEBUG")
logger.add_file_handler("debug.log")

# Example logging session
with logger.session("debug_test") as log:
    # Log different levels
    log.debug("Detailed information")
    log.info("General information")
    log.warning("Warning message")
    log.error("Error message")
    
    try:
        # Simulate error
        raise ValueError("Test error")
    except Exception as e:
        # Log exception with traceback
        log.exception("An error occurred")

# Show log contents
print("\nLog Contents:")
print(logger.get_logs())

# Generate log report
report = logger.generate_report()
print("\nLog Report:")
print(report)

## Error Reporting

Demonstrate ADPA's error reporting capabilities.

In [ ]:
# Configure error reporting
reporter = debugger.get_reporter()
reporter.configure(
    include_vars=True,
    include_trace=True,
    include_context=True
)

# Example error report
try:
    query = "Show me employee details where salary > 'high'"
    result = text2sql.execute(query)
except Exception as e:
    # Generate error report
    report = reporter.generate_report(e)
    
    print("Error Report:")
    print(report.summary)
    
    print("\nDetailed Information:")
    print(f"Error Type: {report.error_type}")
    print(f"Location: {report.location}")
    print(f"Stack Trace: {report.stack_trace}")
    
    print("\nContext:")
    print(f"Query: {report.context['query']}")
    print(f"Variables: {report.context['variables']}")
    
    print("\nSuggestions:")
    for suggestion in report.suggestions:
        print(f"- {suggestion}")

## Cleanup

Clean up resources.

In [ ]:
# Close database connection
db.close()

# Cleanup debugging tools
debugger.cleanup()
tracer.cleanup()

# Close logger
logger.close()